In [ ]:
# !pip install flask_cors
# !pip install flask
# !pip install waitress

In [1]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from flask import Flask
from flask import jsonify, request
from flask_cors import CORS

In [2]:
def load_model_file(filename):
    obj = pickle.load(open(filename, 'rb'))
    print(obj)
    
    model = obj['model']
    vectorizer = obj['vectorizer']
    
    return model, vectorizer

def predict(sentences, model, vectorizer):
    sentences = sentences if isinstance(sentences, list) else [sentences]
    vector = vectorizer.transform(sentences)
    return model.predict(vector)

In [3]:
model, vectorizer = load_model_file('movie_model.pkl')

{'model': LogisticRegression(solver='liblinear'), 'vectorizer': TfidfVectorizer(max_features=3000, stop_words='english')}


In [4]:
print(predict('this is a great movie.', model, vectorizer))

['positive']


In [5]:
print(predict('this is a horrible movie.', model, vectorizer))

['negative']


In [6]:
predict([
    'funny movie',
    'awesome movie',
    'stupid movie'
], model, vectorizer)

array(['positive', 'positive', 'negative'], dtype=object)

In [7]:
app = Flask(__name__)
CORS(app)

@app.route('/')
def start():
    sentence = request.args.get('sentence')

    if sentence == None: 
        return jsonify({})

    # print(sentence)
    
    result = predict(sentence, model, vectorizer)
    
    # print(result)
    return jsonify({
        'sentence': sentence,
        'sentiment': result[0]
    })


if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Dec/2022 19:44:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 19:44:54] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Dec/2022 19:45:21] "GET /?sentence=this%20was%20a%20good%20movie HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 19:45:59] "GET /?sentence=3%20idiots%20is%20my%20favorite%20movie HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 19:46:13] "GET /?sentence=rrr%20denies%20physics HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 19:47:26] "GET /?sentence=it's%20an%20awesome%20movie HTTP/1.1" 200 -
127.0.0.1 - - [06/Dec/2022 19:47:55] "GET /?sentence=fell%20asleep HTTP/1.1" 200 -


## For production

In [8]:
from flask import Flask
from flask import jsonify, request
from flask_cors import CORS
from waitress import serve

app = Flask(__name__)
CORS(app)

@app.route('/')
def start():
    sentence = request.args.get('sentence')
    
    if sentence == None: 
        return jsonify({})

    # print(sentence)
    
    result = predict(sentence, model, vectorizer)
    
    # print(result)
    return jsonify({
        'sentence': sentence,
        'sentiment': result[0]
    })

if __name__ == '__main__':
    serve(app, host="0.0.0.0", port=80)

In [9]:
!ls

IMDB Dataset.csv      movie_model.pkl       setting_up_server.txt
consuming_api.ipynb   server.ipynb          user-data.txt
ml_training.ipynb     server.py


In [10]:
!cat server.py

#!/usr/bin/env python
# coding: utf-8

import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

from flask import Flask
from flask import jsonify, request
from flask_cors import CORS
from waitress import serve

def load_model_file(filename):
    obj = pickle.load(open(filename, 'rb'))
    
    model = obj['model']
    vectorizer = obj['vectorizer']
    
    return model, vectorizer
    return obj

def predict(sentences, model, vectorizer):
    sentences = sentences if isinstance(sentences, list) else [sentences]
    vector = vectorizer.transform(sentences)
    return model.predict(vector)

model, vectorizer = load_model_file('movie_model.pkl')

print(predict('this is a great movie.', model, vectorizer))
print(predict('this is a horrible movie.', model, vectorizer))

app = Flask(__name__)
CORS(app)

@app.route('/')
def start():
    sentence = request.args.get('sentence')

    if sentence == None: 
        return jsoni

In [11]:
!aws s3 cp movie_model.pkl s3://com.msarica.ds/movie_model.pkl
!aws s3 cp server.py s3://com.msarica.ds/server.py

upload: ./movie_model.pkl to s3://com.msarica.ds/movie_model.pkl
upload: ./server.py to s3://com.msarica.ds/server.py           
